# The Art of Prolog
1986年MIT Pressから出版されたPrologの教科書 The Art of Prolog は、私の前職である株式会社 構造計画研究所から1988年1月1日に「Prologの技芸」というタイトルで日本語訳が出版されました。

残念ながらPrologの技芸は絶版となり、Amazonの古本でも12,800円と高額で販売されています。

幸運なことに、英語の原本の方は、MIT Book pressのサーバからダウンロードすることができます。
- https://mitp-content-server.mit.edu/books/content/sectbyfn/books_pres_0/1407/1407.pdf?dl=1

2024年になって、推論エンジンの原理と応用を再度確認するために、「Prologの技芸」の例題をpythonにポーティングした推論エンジンと簡易パーサを使ってトレースしました。

さらに、swi prologを使って、22章の「方程式を解く」と23章の「コンパイラ」をトレースしました。以下のノートはその時の記録に加筆し、整理したものです。

## ノートブック実行の注意
最初に、コマンドパレットを表示し、"Select Notebook Kernel"を実行し、"Prolog"を選択します。
これで、code cellの内容がswi prologによって処理されます。

### 注意
同じfactor名を持つ定義は、同一セル内で記述しないとうまく処理されない！

## 22章　方程式を解く
22章の「方程式を解く」は、数式処理システムPRESSの簡易版として紹介されています。

ここでは、22章の内容をswi-prologを使って動作を確認していきます。

### 因数分解
最初に未知数の位置を取り出します。

In [3]:
position(Term, Term, []) :- !.
% 訳本の記載ミス（原文に合わせた）
position(Sub, Term, Path) :-
    compound(Term), functor(Term, F, N), position(N, Sub, Term, Path), !.
position(N, Sub, Term, [N|Path]) :-
    arg(N, Term, Arg), position(Sub, Arg, Path).
position(N, Sub, Term, Path) :-
    N > 1, N1 is N - 1, position(N1, Sub, Term, Path).


% Asserting clauses for user:position/3


% Asserting clauses for user:position/4


In [4]:
?- position(x, cos(x) = 0, Path).

Path = [1,1]

In [5]:
?- position(x, 1 - 2*sin(x) = 0, Path).

Path = [1,2,2,1]

### 方程式に未知数の数
孤立化を適応するための条件は、方程式中に未知数がXが１度だけ出現することなので、single_occurrenceでその条件を確認します。

In [6]:
single_occurrence(Subterm, Term) :- occurrence(Subterm, Term, 1).
% Term中
occurrence(Term, Term, 1) :- !.
occurrence(Sub, Term, N) :-
    compound(Term), !, functor(Term, F, M), occurrence(M, Sub, Term, 0, N).
% 原本に合わせる
occurrence(Sub, Term, 0) :- Term \= Sub.
occurrence(M, Sub, Term, N1, N2) :-
    M > 0, !, arg(M, Term, Arg), occurrence(Sub, Arg, N), N3 is N + N1,
    M1 is M - 1, occurrence(M1, Sub, Term, N3, N2).
occurrence(0, Sub, Term, N, N).

% Asserting clauses for user:single_occurrence/2


% Asserting clauses for user:occurrence/3


% Asserting clauses for user:occurrence/5


In [7]:
?- single_occurrence(x, cos(x) = 0).

true

maneuver_sideで未知数Xが方程式の左辺に位置するようにします。

In [8]:
maneuver_side(1, Lhs = Rhs, Lhs = Rhs) :- !.
maneuver_side(2, Lhs = Rhs, Rhs = Lhs) :- !.

% Asserting clauses for user:maneuver_side/3


### 孤立化

In [9]:
isolate([N|Position], Equation, IsolatedEquation) :-
    isolax(N, Equation, Equation1),
    isolate(Position, Equation1, IsolatedEquation).
isolate([], Equation, Equation).

% Asserting clauses for user:isolate/3


孤立化で使用する公理

In [10]:
isolax(1, -Lhs = Rhs, Lhs = -Rhs).
isolax(1, Term1 + Term2 = Rhs, Term1 = Rhs - Term2).
isolax(2, Term1 + Term2 = Rhs, Term2 = Rhs - Term1).
isolax(1, Term1 - Term2 = Rhs, Term1 = Rhs + Term2).
isolax(2, Term1 - Term2 = Rhs, Term2 = Term1 - Rhs).
isolax(1, Term1*Term2 = Rhs, Term1 = Rhs/Term2) :- Term2 \= 0.
isolax(2, Term1*Term2 = Rhs, Term2 = Rhs/Term1) :- Term1 \= 0.
isolax(1, Term1^Term2 = Rhs, Term1 = Rhs^(-Term2)).
isolax(2, Term1^Term2 = Rhs, Term2 = log(base(Term1), Rhs)).
isolax(1, sin(U) = V, U = arcsin(V)).
isolax(1, sin(U) = V, U = pi - arcsin(V)).
isolax(1, cos(U) = V, U = arccos(V)).
isolax(1, cos(U) = V, U = -arccos(V)).

% Asserting clauses for user:isolax/3


### 孤立化の解


In [11]:
solve_equation_isolate(Equation, X, Solution) :-
    single_occurrence(X, Equation),
    !,
    position(X, Equation, [Side|Position]),
    maneuver_side(Side, Equation, Equation1),
    isolate(Position, Equation1, Solution).

% Asserting clauses for user:solve_equation_isolate/3


先の例で挙げられたcos(x) = 0を解いてみます。

In [12]:
?- solve_equation_isolate(cos(x) = 0, x, Solution).
?- retry.

Solution = x=arccos(0)

% Retrying goal: solve_equation_isolate(cos(x)=0,x,Solution)


Solution = x= -arccos(0)

同様に1 - 2*sin(x) = 0を解いてみます。

In [13]:
?- solve_equation_isolate(1 - 2*sin(x) = 0, x, Solution).
?- retry.

Solution = x=arcsin((1-0)/2)

% Retrying goal: solve_equation_isolate(1-2*sin(x)=0,x,Solution)


Solution = x=pi-arcsin((1-0)/2)

## 因数分解法の解

In [14]:
factorize(A*B, X, Factors, Rest) :-
    !, factorize(A, X, Factors, Factors1),
    factorize(B, X, Factors1, Rest).
factorize(C, X, [C|Factors], Factors) :-
    subterm(X, C), !.
factorize(C, X, Factors, Factors).

subterm(Term, Term).
subterm(Sub, Term) :-
    compound(Term), functor(Term, F, N), subterm(N, Sub, Term).
subterm(N, Sub, Term) :-
    N > 1, N1 is N - 1, subterm(N1, Sub, Term).
subterm(N, Sub, Term) :-
    arg(N, Term, Arg), subterm(Sub, Arg).

% Asserting clauses for user:factorize/4


% Asserting clauses for user:subterm/2


% Asserting clauses for user:subterm/3


In [15]:
solve_equation(A*B = 0, X, Solution) :-
    !,
    factorize(A*B, X, Factors, []),
    remove_duplicates(Factors, Factors1),
    solve_factors(Factors1, X, Solution).

% Asserting clauses for user:solve_equation/3


In [16]:
solve_factors([Factor|Factors], X, Solution) :-
    solve_equation_isolate(Factor = 0, X, Solution).
solve_factors([Factor|Factors], X, Solutions) :-
    solve_factors(Factors, X, Solutions).

remove_duplicates([], []).
remove_duplicates([X|Xs], [X|Ys]) :-
    remove_duplicates(Xs, Ys).
remove_duplicates([X|Xs], Ys) :-
    member(X, Xs).
remove_duplicates(Xs, Ys).

% Asserting clauses for user:solve_factors/3


% Asserting clauses for user:remove_duplicates/2


### 動作確認
最初にfactorizeの確認をします。

In [17]:
?- factorize(cos(x)*(1 - 2*sin(x)), x, Factors, []).

Factors = [cos(x),1-2*sin(x)]

次にsolve_factorsの動作を確認します。

In [18]:
?- solve_factors([cos(x), 1 - 2*sin(x)], x, Solution).
?- retry.
?- retry.
?- retry.

Solution = x=arccos(0)

% Retrying goal: solve_factors([cos(x),1-2*sin(x)],x,Solution)


Solution = x= -arccos(0)

% Retrying goal: solve_factors([cos(x),1-2*sin(x)],x,Solution)


Solution = x=arcsin((1-0)/2)

% Retrying goal: solve_factors([cos(x),1-2*sin(x)],x,Solution)


Solution = x=pi-arcsin((1-0)/2)

最後にsolve_equationの動作で、動作を確認します。

In [19]:
?- solve_equation(cos(x)*(1-2*sin(x)) = 0, x, Solution).
?- retry.
?- retry.
?- retry.

Solution = x=arccos(0)

% Retrying goal: solve_equation(cos(x)*(1-2*sin(x))=0,x,Solution)


Solution = x= -arccos(0)

% Retrying goal: solve_equation(cos(x)*(1-2*sin(x))=0,x,Solution)


Solution = x=arcsin((1-0)/2)

% Retrying goal: solve_equation(cos(x)*(1-2*sin(x))=0,x,Solution)


Solution = x=pi-arcsin((1-0)/2)

### 因数分解の処理について
孤立化のsolve_equationと因数分解のsolve_equationを同じ名前にすると正しく動作しないため、孤立化のsolve_equationをsolve_eauation_isolateに変更しました。
（注: 後に同じ名前の複数のルールは同一のセルで定義しないとうまく処理できないことが分かり、それが原因だと思われる）

## 多項式

多項式か否かを判断するpolynomialは、以下のように定義されます。

```prolog
polynomial(Term, X) :-
    Termは、Xの多項式である。
```

In [20]:
polynomial(X, X) :- !.
polynomial(Term, X) :- constant(Term), !.
polynomial(Term1 + Term2, X) :-
    !, polynomial(Term1, X), polynomial(Term2, X).
polynomial(Term1 - Term2, X) :-
    !, polynomial(Term1, X), polynomial(Term2, X).
polynomial(Term1 * Term2, X) :-
    !, polynomial(Term1, X), polynomial(Term2, X).
polynomial(Term1 / Term2, X) :-
    !, polynomial(Term1, X), constant(Term2).
polynomial(Term ^ N, X) :-
    !, natural_number(N),  polynomial(Term, X).

% 本に定義がないので、constantとnatural_numberを以下のように定義
constant(N) :- number(N).
natural_number(0).
natural_number(N) :- number(N), N >= 0, M is N - 1, natural_number(M).

% Asserting clauses for user:polynomial/2


% Asserting clauses for user:constant/1


% Asserting clauses for user:natural_number/1


### 多項式を配列に変換
polynomial_formは、多項式を(係数, 次数)の配列に変換します。
```prolog
polynomial_form(X^N, X, [(1, N)])
```

In [21]:
% 定義に定数の場合が抜けていた
polynomial_form(C, X, [(C, 0)]) :- constant(C).
% 定数割に対応
polynomial_form(A/C, X, [(A/C, 0)]) :- constant(A), constant(C), C \= 0.
polynomial_form(X/C, X, [(1/C, 1)]) :- constant(C), C \= 0.
% 以下原著から抜粋
polynomial_form(X, X, [(1, 1)]).
polynomial_form(X^N, X, [(1, N)]).
polynomial_form(Term1 + Term2, X, PolyForm) :-
    polynomial_form(Term1, X, PolyForm1),
    polynomial_form(Term2, X, PolyForm2),
    add_polynomials(PolyForm1, PolyForm2, PolyForm).
polynomial_form(Term1 - Term2, X, PolyForm) :-
    polynomial_form(Term1, X, PolyForm1),
    polynomial_form(Term2, X, PolyForm2),
    subtract_polynomials(PolyForm1, PolyForm2, PolyForm).
polynomial_form(Term1 * Term2, X, PolyForm) :-
    polynomial_form(Term1, X, PolyForm1),
    polynomial_form(Term2, X, PolyForm2),
    multiply_polynomials(PolyForm1, PolyForm2, PolyForm).
% 定数割を追加
polynomial_form(Term/C, X, PolyForm) :-
    constant(C), C \= 0,
    polynomial_form(Term, X, PolyForm1),
    multiply_polynomials(PolyForm1, [(1/C, 0)], PolyForm).
polynomial_form(Term ^ N, X, PolyForm) :- !, Term \= X,
polynomial_form(Term, X, PolyForm1),
binominal(PolyForm1, N, PolyForm).

binominal(Poly, 1, Poly).
% 原著に定義ないので、以下のように定義
binominal(Poly, N, PolyN) :-
    number(N),
    N > 1, 
    N1 is N - 1, 
    binominal(Poly, N1, PolyN1),
    multiply_polynomials(PolyN1, Poly, PolyN).

% Asserting clauses for user:polynomial_form/3


% Asserting clauses for user:binominal/3


In [22]:
add_polynomials([], Poly, Poly) :- !.
add_polynomials(Poly, [], Poly) :- !.
add_polynomials([(Ai, Ni)|Poly1], [(Aj, Nj)|Poly2], [(Ai, Ni)|Poly]):-
    Ni > Nj, add_polynomials(Poly1, [(Aj, Nj)|Poly2], Poly).
add_polynomials([(Ai, Ni)|Poly1], [(Aj, Nj)|Poly2], [(A, Ni)|Poly]):-
    Ni =:= Nj, A is Ai + Aj, add_polynomials(Poly1, Poly2, Poly).
add_polynomials([(Ai, Ni)|Poly1], [(Aj, Nj)|Poly2], [(Aj, Nj)|Poly]):-
    Ni < Nj, add_polynomials([(Ai, Ni)|Poly1], Poly2, Poly).

subtract_polynomials(Poly1, Poly2, Poly) :-
    multiply_single(Poly2, (-1, 0), Poly3),
    add_polynomials(Poly1, Poly3, Poly), !. 

multiply_single([(C1, N1)|Poly1], (C, N), [(C2, N2)|Poly]) :-
    C2 is C1*C, N2 is N1 + N, multiply_single(Poly1, (C, N), Poly).
multiply_single([], Factor, []).

multiply_polynomials([(C, N)|Poly1], Poly2, Poly) :-
    multiply_single(Poly2, (C, N), Poly3),
    multiply_polynomials(Poly1, Poly2, Poly4),
    add_polynomials(Poly3, Poly4, Poly).
multiply_polynomials([], P, []).

% Asserting clauses for user:add_polynomials/3


% Asserting clauses for user:subtract_polynomials/3


% Asserting clauses for user:multiply_single/3


% Asserting clauses for user:multiply_polynomials/3


### polynomial_formの動作確認

ｘの2次元多項式 x^2 - 3*x +2 をpolynomial_formで(係数, 次数)の配列に変換します。

In [23]:
?- polynomial_form((x^2 - 3*x + 2), x, Poly).

Poly = [(1,2),(-3,1),(2,0)]

### 多項式の累乗の確認
最初にbinominalの確認をします。

同次の項をまとめていませんが、結果は正しく行われています。

In [24]:
?- binominal([(1, 0), (1, 1)], 2, Poly).

Poly = [(1,1),(1,2),(1,0),(1,1)]

次に、polynominal_formを実行すると、同次項がまとめられ、降順で表示された。

In [25]:
?- polynomial_form(((1 + x)^2), x, Poly).

Poly = [(1,2),(2,1),(1,0)]

### 多項式標準系
係数が０の項を削除し、簡潔な多項式形式を出力するpolynomial_normal_formを定義します。

In [26]:
polynomial_normal_form(Polynominal, X, NormalForm) :-
    polynomial_form(Polynominal, X, PolyForm),
    remove_zero_terms(PolyForm, NormalForm), !.

remove_zero_terms([(0, N)|Poly], Poly1) :-
    !, remove_zero_terms(Poly, Poly1).
remove_zero_terms([(C, N)|Poly], [(C, N)|Poly1]) :-
    C \= 0, !, remove_zero_terms(Poly, Poly1).
remove_zero_terms([], []).

% Asserting clauses for user:polynomial_normal_form/3


% Asserting clauses for user:remove_zero_terms/2


In [27]:
polynomial_form((x^2 - 3*x + 3*x + 2), x, Poly).

Poly = [(1,2),(0,1),(2,0)]

In [28]:
?- polynomial_normal_form((x^2 - 3*x + 3*x + 2), x, Poly).

Poly = [(1,2),(2,0)]

### 多項式標準系から１次方程式、2次方程式の解を求める
solve_polynomial_equationに多項式標準系から１次方程式、2次方程式の解を求める処理を定義します。


In [29]:
solve_polynomial_equation(PolyEquation, X, X = -B/A) :-
    linear(PolyEquation), !,
    pad(PolyEquation, [(A, 1), (B, 0)]).
solve_polynomial_equation(PolyEquation, X, Solution) :-
    quadratic(PolyEquation),
    pad(PolyEquation, [(A, 2), (B, 1), (C, 0)]),
    discriminant(A, B, C, Discriminant),
    root(X, A, B, C, Discriminant, Solution).
% 1次の判定は、先頭が(Coeff, 1)であること
linear([(Coeff, 1)|Poly1]).
% 2次の判定は、先頭が(Coeff, 2)であること
quadratic([(Coeff, 2)|Poly1]).
% padは、各次数の係数を埋める
pad([(C, N)|Poly], [(C, N)|Poly1]):-
    !, pad(Poly, Poly1).
pad(Poly, [(0, N)|Poly1]) :-
    pad(Poly, Poly1).
pad([], []).
% 判別式descriminantを計算
discriminant(A, B, C, D) :- D is B*B - 4*A*C.
% 根の計算
root(X, A, B, C, 0, X = -B/(2*A)).
root(X, A, B, C, D, X = (-B+sqrt(D))/(2*A)) :- D > 0.
root(X, A, B, C, D, X = (-B-sqrt(D))/(2*A)) :- D > 0.

% Asserting clauses for user:solve_polynomial_equation/3


% Asserting clauses for user:linear/1


% Asserting clauses for user:quadratic/1


% Asserting clauses for user:pad/2


% Asserting clauses for user:discriminant/4


% Asserting clauses for user:root/6


### solve_polynomial_equationの動作確認
最初に一次の方程式の解を確認します。

In [30]:
solve_equation_poly(Lhs = Rhs, X, Solution) :-
    polynomial(Lhs, X),
    polynomial(Rhs, X),
    !,
    polynomial_normal_form(Lhs-Rhs, X, PolyForm),
    solve_polynomial_equation(PolyForm, X, Solution).

% Asserting clauses for user:solve_equation_poly/3


In [31]:
?- solve_equation_poly((2*x - 1) = 0, x, Solution)

Solution = x= - -1/2

次に2次方程式の解を確認します。

In [32]:
?- solve_equation_poly((x^2 - 3*x + 2) = 0, x, Solution).
?- retry.

Solution = x=(- -3+sqrt(1))/(2*1)

% Retrying goal: solve_equation_poly(x^2-3*x+2=0,x,Solution)


Solution = x=(- -3-sqrt(1))/(2*1)

## 同次化

- 背反項集合（offenders set）: 未知数を含む最大の非多項式項の集合
- 既約項（reduced term）

In [33]:
homogenize(Equation, X, Equation1, X1) :-
    offenders(Equation, X, Offenders),
    reduced_term(X, Offenders, Type, X1),
    rewrite(Offenders, Type, X1, Substitutions),
    substitute(Equation, Substitutions, Equation1).


% Asserting clauses for user:homogenize/4


### 背反項集合の抽出（offenders）
背反項集合の抽出するルール（offenders）は、以下のようになります。

In [34]:
offenders(Equation, X, Offenders) :- 
    parse(Equation, X, Offenders1, []),
    % 訳本の定義が原本と異なる
    % remove_term(Offenders1, Offenders).
    remove_duplicates(Offenders1, Offenders),
    multiple(Offenders).

parse(A + B, X, L1, L2) :-
    !, parse(A, X, L1, L3), parse(B, X, L3, L2).
parse(A * B, X, L1, L2) :-
    !, parse(A, X, L1, L3), parse(B, X, L3, L2).
parse(A - B, X, L1, L2) :-
    !, parse(A, X, L1, L3), parse(B, X, L3, L2).
parse(A = B, X, L1, L2) :-
    !, parse(A, X, L1, L3), parse(B, X, L3, L2).
parse(A ^ B, X, L, []) :-
    integer(B), !, parse(A, X, L, []).
parse(A, X, L, L) :-
    free_of(X, A), !.
parse(A, X, [A|L], L) :-
    subterm(X, A), !.

free_of(Subterm, Term) :-
    occurrence(Subterm, Term, N), !, N = 0.

remove_duplicates([], []).
remove_duplicates([X|Xs], [X|Ys]) :-
    remove_duplicates(Xs, Ys).
remove_duplicates([X|Xs], Ys) :-
    member(X, Xs).
% 意味がわからない
multiple([X1, X2|Xs]).

% Asserting clauses for user:offenders/3


% Asserting clauses for user:parse/4


% Asserting clauses for user:free_of/2


Previously defined clauses of user:remove_duplicates/2 were retracted:
:- dynamic remove_duplicates/2.

remove_duplicates([], []).
remove_duplicates([A|B], [A|C]) :-
    remove_duplicates(B, C).
remove_duplicates([A|B], _) :-
    member(A, B).
remove_duplicates(_, _).

% Asserting clauses for user:remove_duplicates/2


% Asserting clauses for user:multiple/1


### 背反項集合（offenders）の動作確認
背反項集合の型は指数（非多項式項だから）で、$A^B$という形式をしており、Aには未知数が含まれず、Bには含まれるものを取り出します。

offendersに$2^{2 \cdot x} - 5 \cdot 2 ^{x + 1} +16 = 0$に対する背反項集合を求めると$\{2^{2 \cdot x}, 2^{x + 1}\}$となります。

In [35]:
?- offenders(2^(2*x) - 5*2^(x + 1) + 16 = 0, x, Offenders).

Offenders = [2^(2*x),2^(x+1)]

In [36]:
reduced_term(X, Offenders, Type, X1) :-
    classify(Offenders, X, Type),
    candidate(Type, Offenders, X, X1).

classify(Offenders, X, exponential) :-
    exponential_offenders(Offenders, X).

exponential_offenders([A^B|Offs], X) :-
    free_of(X, A), subterm(X, B), exponential_offenders(Offs, X).
exponential_offenders([], X).

candidate(exponential, Offenders, X, A^X) :-
    base(Offenders, A), polynomial_exponents(Offenders, X).

base([A^B|Offs], A) :- base(Offs, A).
base([], A).

polynomial_exponents([A^B|Offs], X) :-
    polynomial(B, X), polynomial_exponents(Offs, X).
polynomial_exponents([], X).

% Asserting clauses for user:reduced_term/4


% Asserting clauses for user:classify/3


% Asserting clauses for user:exponential_offenders/2


% Asserting clauses for user:candidate/4


% Asserting clauses for user:base/2


% Asserting clauses for user:polynomial_exponents/2


### candidateの動作確認


In [37]:
?- candidate(exponential, [2^(2*x),2^(x+1)], x, X1).

X1 = 2^x

In [38]:
?- reduced_term(x, [2^(2*x),2^(x+1)], Type, X1).

Type = exponential,
X1 = 2^x

### 同次化書き換え
指数関数の置き換え規則（homog_axiom）は、以下の3つです。
- $A^(N*X)$は$A^X$を使って$(A^X)^N$に置換
- $A^(-X)$は、$A^X$を使って1/(A^X)$に置換
- $A^(X + B)$は、$A^X$を使って$A^B \cdot A^X$に置換

In [39]:
rewrite([Off|Offs], Type, X1, [Off = Term|Rewrites]) :-
    homog_axiom(Type, Off, X1, Term),
    rewrite(Offs, Type, X1, Rewrites).
rewrite([], Type, X, []).

% 同次公理
homog_axiom(exponential, A^(N*X), A^X, (A^X)^N).
homog_axiom(exponential, A^(-X), A^X, 1/(A^X)).
homog_axiom(exponential, A^(X+B), A^X, A^B*A^X).

% Asserting clauses for user:rewrite/4


% Asserting clauses for user:homog_axiom/4


In [40]:
?- rewrite([2^(2*x),2^(x+1)], exponential, 2^x, Substitutions).

Substitutions = [2^(2*x)=(2^x)^2,2^(x+1)=2^1*2^x]

## 式の置換


In [41]:
substitute(A + B, Subs, NewA + NewB) :-
    !, substitute(A, Subs, NewA), substitute(B, Subs, NewB).
substitute(A * B, Subs, NewA * NewB) :-
    !, substitute(A, Subs, NewA), substitute(B, Subs, NewB).
substitute(A - B, Subs, NewA - NewB) :-
    !, substitute(A, Subs, NewA), substitute(B, Subs, NewB).
substitute(A = B, Subs, NewA = NewB) :-
    !, substitute(A, Subs, NewA), substitute(B, Subs, NewB).
substitute(A ^ B, Subs, NewA ^ B) :-
    integer(B), !, substitute(A, Subs, NewA).
substitute(A, Subs, B) :-
    member(A=B, Subs), !.
substitute(A/A, Subs, 1) :- A \= 0.
substitute(A, Subs, A).

% Asserting clauses for user:substitute/3


In [42]:
?- substitute(2^(2*x) - 5*2^(x + 1) + 16 = 0, [2^(2*x)=(2^x)^2,2^(x+1)=2^1*2^x], Eq).

Eq = (2^x)^2-5*(2^1*2^x)+16=0

In [43]:
?- homogenize(2^(2*x) - 5*2^(x + 1) + 16 = 0, x, Equation1, X1).

Equation1 = (2^x)^2-5*(2^1*2^x)+16=0,
X1 = 2^x

## solve_equationの再定義
prologカーネルでは、同じfactorの定義は同一セルで行わないと正常に処理できないみたいなので、solve_equationを再度定義し直して、これまでの処理を一つにまとめます。

In [44]:
solve_equation(A*B=0, X, Solution) :-
    !,
    factorize(A*B, X, Factors, []),
    remove_duplicates(Factors, Factors1),
    solve_factors(Factors, X, Solution).
solve_equation(Equation, X, Solution) :-
    single_occurrence(X, Equation),
    !,
    position(X, Equation, [Side|Position]),
    maneuver_side(Side, Equation, Equation1),
    isolate(Position, Equation1, Solution).
solve_equation(Lhs=Rhs, X, Solution) :-
    polynomial(Lhs, X),
    polynomial(Rhs, X),
    !,
    polynomial_normal_form(Lhs - Rhs, X, PolyForm),
    solve_polynomial_equation(PolyForm, X, Solution).
solve_equation(Equation, X, Solution) :-
    homogenize(Equation, X, Equation1, X1),
    !,
    solve_equation(Equation1, X1, Solution1),
    solve_equation(Solution1, X, Solution).


Previously defined clauses of user:solve_equation/3 were retracted:
:- dynamic solve_equation/3.

solve_equation(A*B=0, C, F) :-
    !,
    factorize(A*B, C, D, []),
    remove_duplicates(D, E),
    solve_factors(E, C, F).

% Asserting clauses for user:solve_equation/3


In [45]:
?- solve_equation(cos(x)*(1 - 2*sin(x)) = 0, x, Solution).
?- retry.
?- retry.
?- retry.

Solution = x=arccos(0)

% Retrying goal: solve_equation(cos(x)*(1-2*sin(x))=0,x,Solution)


Solution = x= -arccos(0)

% Retrying goal: solve_equation(cos(x)*(1-2*sin(x))=0,x,Solution)


Solution = x=arcsin((1-0)/2)

% Retrying goal: solve_equation(cos(x)*(1-2*sin(x))=0,x,Solution)


Solution = x=pi-arcsin((1-0)/2)

In [46]:
?- solve_equation(1 - 2*sin(x) = 0, x, Solution).
?- retry.

Solution = x=arcsin((1-0)/2)

% Retrying goal: solve_equation(1-2*sin(x)=0,x,Solution)


Solution = x=pi-arcsin((1-0)/2)

In [47]:
?- solve_equation(x^2 - 3*x + 2 = 0, x, Solution).
?- retry.

Solution = x=(- -3+sqrt(1))/(2*1)

% Retrying goal: solve_equation(x^2-3*x+2=0,x,Solution)


Solution = x=(- -3-sqrt(1))/(2*1)

In [48]:
?- solve_equation(2^(2*x) - 5*2^(x + 1) + 16 = 0, x, Solution).
?- retry.

Solution = x=log(base(2),(- -10+sqrt(36))/(2*1))

% Retrying goal: solve_equation(2^(2*x)-5*2^(x+1)+16=0,x,Solution)


Solution = x=log(base(2),(- -10-sqrt(36))/(2*1))

## 練習問題

### 1 左辺の除算追加
最初に、現行の定義では、$x/2 = 5$が解けないことを確認します。

In [49]:
% 下記のコメントを外してください
% ?- solve_equation(x/2 = 5, x, Solution).

次にisolaxに$\frac{Term1}{Term2} = Rhs$の公理を追加します。


In [ ]:
isolax(1, -Lhs = Rhs, Lhs = -Rhs).
isolax(1, Term1 + Term2 = Rhs, Term1 = Rhs - Term2).
isolax(2, Term1 + Term2 = Rhs, Term2 = Rhs - Term1).
isolax(1, Term1 - Term2 = Rhs, Term1 = Rhs + Term2).
isolax(2, Term1 - Term2 = Rhs, Term2 = Term1 - Rhs).
isolax(1, Term1*Term2 = Rhs, Term1 = Rhs/Term2) :- Term2 \= 0.
isolax(2, Term1*Term2 = Rhs, Term2 = Rhs/Term1) :- Term1 \= 0.
isolax(1, Term1^Term2 = Rhs, Term1 = Rhs^(-Term2)).
isolax(2, Term1^Term2 = Rhs, Term2 = log(base(Term1), Rhs)).
isolax(1, sin(U) = V, U = arcsin(V)).
isolax(1, sin(U) = V, U = pi - arcsin(V)).
isolax(1, cos(U) = V, U = arccos(V)).
isolax(1, cos(U) = V, U = -arccos(V)).
% 練習問題1用追加公理
isolax(1, Term1/Term2 = Rhs, Term1 = Rhs * Term2) :- Term2 \= 0.
isolax(2, Term1/Term2 = Rhs, Term2 = Term1/Rhs) :- Rhs \= 0.

Previously defined clauses of user:isolax/3 were retracted:
:- dynamic isolax/3.

isolax(1, -A=B, A= -B).
isolax(1, A+C=B, A=B-C).
isolax(2, C+A=B, A=B-C).
isolax(1, A-C=B, A=B+C).
isolax(2, B-A=C, A=B-C).
isolax(1, A*C=B, A=B/C) :-
    C\=0.
isolax(2, C*A=B, A=B/C) :-
    C\=0.
isolax(1, A^C=B, A=B^ -C).
isolax(2, B^A=C, A=log(base(B), C)).
isolax(1, sin(A)=B, A=arcsin(B)).
isolax(1, sin(A)=B, A=pi-arcsin(B)).
isolax(1, cos(A)=B, A=arccos(B)).
isolax(1, cos(A)=B, A= -arccos(B)).

% Asserting clauses for user:isolax/3


In [51]:
?- solve_equation(2/x = 5, x, Solution).

Solution = x=2/5

In [52]:
?- solve_equation(x^3 = 1, x, Solution).

Solution = x=1^ - 3

### 3. 三角関数の処理
offendersに三角関数を与えると以下のように非線形項に追加されます。

In [53]:
?- offenders(cos(2*x) - sin(x) = 0, x, Offenders).

Offenders = [cos(2*x),sin(x)]

In [54]:
reduced_term(X, Offenders, Type, X1) :-
    classify(Offenders, X, Type),
    candidate(Type, Offenders, X, X1).

classify(Offenders, X, exponential) :-
    exponential_offenders(Offenders, X).

exponential_offenders([A^B|Offs], X) :-
    free_of(X, A), subterm(X, B), exponential_offenders(Offs, X).
exponential_offenders([], X).

candidate(exponential, Offenders, X, A^X) :-
    base(Offenders, A), polynomial_exponents(Offenders, X).

base([A^B|Offs], A) :- base(Offs, A).
base([], A).

polynomial_exponents([A^B|Offs], X) :-
    polynomial(B, X), polynomial_exponents(Offs, X).
polynomial_exponents([], X).

% 三角関数
classify(Offenders, X, trigonometric) :-
    trigonometric_offenders(Offenders, X).
% 三角関数を含む項
trigonometric_offenders([cos(A)|Offs], X) :-
    subterm(X, A), trigonometric_offenders(Offs, X).
trigonometric_offenders([sin(A)|Offs], X) :-
    subterm(X, A), trigonometric_offenders(Offs, X).
trigonometric_offenders([], X).
% 置換の対象は、sin(x), cos(x)の基底関数とする
candidate(trigonometric, Offenders, X, X1) :-
    trigonometric_base(X, Offenders, X1).
% sin(x), cos(x)のみを抽出
is_trigonometric_base(X, A) :- A == cos(X).
is_trigonometric_base(X, A) :- A == sin(X).
trigonometric_base(X, In, X1) :-
    include(is_trigonometric_base(X), In, [X1]).

Previously defined clauses of user:reduced_term/4 were retracted:
:- dynamic reduced_term/4.

reduced_term(B, A, C, D) :-
    classify(A, B, C),
    candidate(C, A, B, D).

% Asserting clauses for user:reduced_term/4


Previously defined clauses of user:classify/3 were retracted:
:- dynamic classify/3.

classify(A, B, exponential) :-
    exponential_offenders(A, B).

% Asserting clauses for user:classify/3


Previously defined clauses of user:exponential_offenders/2 were retracted:
:- dynamic exponential_offenders/2.

exponential_offenders([B^C|D], A) :-
    free_of(A, B),
    subterm(A, C),
    exponential_offenders(D, A).
exponential_offenders([], _).

...

Previously defined clauses of user:polynomial_exponents/2 were retracted:
:- dynamic polynomial_exponents/2.

polynomial_exponents([_^A|C], B) :-
    polynomial(A, B),
    polynomial_exponents(C, B).
polynomial_exponents([], _).

% Asserting clauses for user:polynomial_exponents/2


% Asserting clauses for user:trigonometric_offenders/2


% Asserting clauses for user:is_trigonometric_base/2


% Asserting clauses for user:trigonometric_base/3


同次公理の追加

In [55]:
% 同次公理
homog_axiom(exponential, A^(N*X), A^X, (A^X)^N).
homog_axiom(exponential, A^(-X), A^X, 1/(A^X)).
homog_axiom(exponential, A^(X+B), A^X, A^B*A^X).

% 変換のないsin(x), cos(x)も含めて定義
homog_axiom(trigonometric, cos(X), cos(X), cos(X)).
homog_axiom(trigonometric, sin(X), sin(X), sin(X)).
% 三角関数倍角定義の追加
homog_axiom(trigonometric, cos(2*X), sin(X), 1 - 2*sin(X)^2).
homog_axiom(trigonometric, sin(2*X), cos(X), 2*sin(X)*cos(X)).

Previously defined clauses of user:homog_axiom/4 were retracted:
:- dynamic homog_axiom/4.

homog_axiom(exponential, A^(C*B), A^B,  (A^B)^C).
homog_axiom(exponential, A^ -B, A^B, 1/A^B).
homog_axiom(exponential, A^(B+C), A^B, A^C*A^B).

% Asserting clauses for user:homog_axiom/4


### $cos(2 \cdot x) - sin(x) = 0$を解く


In [56]:
?- offenders(cos(2*x) - sin(x) = 0, x, Offenders).
?- classify([cos(2*x),sin(x)], x, Type).
?- homogenize(cos(2*x) - sin(x) = 0, x, Equation1, X1).


Offenders = [cos(2*x),sin(x)]

Type = trigonometric

Equation1 = 1-2*sin(x)^2-sin(x)=0,
X1 = sin(x)

In [57]:
?- solve_equation(cos(2*x) - sin(x) = 0, x, Solution).
?- retry.
?- retry.
?- retry.

Solution = x=arcsin((- -1+sqrt(9))/(2* -2))

% Retrying goal: solve_equation(cos(2*x)-sin(x)=0,x,Solution)


Solution = x=pi-arcsin((- -1+sqrt(9))/(2* -2))

% Retrying goal: solve_equation(cos(2*x)-sin(x)=0,x,Solution)


Solution = x=arcsin((- -1-sqrt(9))/(2* -2))

% Retrying goal: solve_equation(cos(2*x)-sin(x)=0,x,Solution)


Solution = x=pi-arcsin((- -1-sqrt(9))/(2* -2))

### 連立方程式を解く

In [58]:
% 各方程式の解を求める
solve_each_equations([Eq|Eqs], [X|Xs], [Sol|Sols]) :-
    solve_equation(Eq, X, Sol), 
    solve_each_equations(Eqs, Xs, Sols).
solve_each_equations([], [], []).

solve_simultaneous_equations([Eq|Eqs], [X|Xs], [Sol2|Sols2]) :-
    solve_each_equations([Eq|Eqs], [X|Xs], [Sol|Sols]), 
    substitute(Sol, Sols, Eq1), 
    solve_equation(Eq1, X, Sol2),
    substitue_solution(Eqs, Sol2, Eqs2), 
    solve_simultaneous_equations(Eqs2, Xs, Sols2).

solve_simultaneous_equations([], [], []).

substitue_solution([Eq|Eqs], Sol, [Eq2|Eqs2]) :-
    substitute(Eq, [Sol], Eq2),
    substitue_solution(Eqs, Sol, Eqs2).
substitue_solution([], Sol, []).

numeric_solve(X = Rhs, X, X = N) :- N is Rhs.

Previously defined clauses of user:solve_each_equations/3 were retracted:
:- dynamic solve_each_equations/3.

solve_each_equations([A|D], [B|E], [C|F]) :-
    solve_equation(A, B, C),
    solve_each_equations(D, E, F).
solve_each_equations([], [], []).

% Asserting clauses for user:solve_each_equations/3


Previously defined clauses of user:solve_simultaneous_equations/3 were retracted:
:- dynamic solve_simultaneous_equations/3.

solve_simultaneous_equations([A|B], [C|D], [H|J]) :-
    solve_each_equations([A|B],
                         [C|D],
                         [E|F]),
    substitute(E, F, G),
    solve_equation(G, C, H),
    substitue_solution(B, H, I),
    solve_simultaneous_equations(I, D, J).
solve_simultaneous_equations([], [], []).

% Asserting clauses for user:solve_simultaneous_equations/3


Previously defined clauses of user:substitue_solution/3 were retracted:
:- dynamic substitue_solution/3.

substitue_solution([A|D], B, [C|E]) :-
    substitute(A, [B], C),
    substitue_solution(D, B, E).
substitue_solution([], _, []).

% Asserting clauses for user:substitue_solution/3


Previously defined clauses of user:numeric_solve/3 were retracted:
:- dynamic numeric_solve/3.

numeric_solve(A=C, A, A=B) :-
    B is C.

% Asserting clauses for user:numeric_solve/3


In [59]:
?- solve_each_equations([x + y = 2, x + 3*y = 4], [x, y], Sols).

Sols = [x=2-y,y=(4-x)/3]

In [60]:
?- solve_simultaneous_equations([x + y = 2, x + 3*y = 4], [x, y], Sols).

Sols = [x= - -0.6666666666666667/0.6666666666666667,y=(4- - -0.6666666666666667/0.6666666666666667)/3]

### 連立方程式の解を計算
連立方程式の解を求めた場合、両辺に変数が出現し、孤立化では解を求めることができません。

occurrenceの戻り値Nが2となります。

In [61]:
?- occurrence(x, x=2-(4-x)/3, N).

N = 2

そこで、多項式で定数項の割り算ができるように修正し、解を求めることにしました。

In [62]:
?- solve_equation(x=2-(4-x)/3, x, Poly).

Poly = x= - -0.6666666666666667/0.6666666666666667

## 解の数値化
１次連立方程式の解は、値が返されるので、numeric_solveを使って、数値化した解を出力するようにします。

In [65]:
?- solve_equation(x=2-(4-x)/3, x, Poly), numeric_solve(Poly, x, Solv).


Poly = x= - -0.6666666666666667/0.6666666666666667,
Solv = x=1.0

In [66]:
% 右辺の数式を評価し、数値に変換する
numeric_solve(X = Rhs, X, X = N) :- N is Rhs.

Previously defined clauses of user:numeric_solve/3 were retracted:
:- dynamic numeric_solve/3.

numeric_solve(A=C, A, A=B) :-
    B is C.

% Asserting clauses for user:numeric_solve/3


In [67]:
% 各方程式の解を求める
solve_each_equations([Eq|Eqs], [X|Xs], [Sol|Sols]) :-
    solve_equation(Eq, X, Sol), 
    solve_each_equations(Eqs, Xs, Sols).
solve_each_equations([], [], []).



Previously defined clauses of user:solve_each_equations/3 were retracted:
:- dynamic solve_each_equations/3.

solve_each_equations([A|D], [B|E], [C|F]) :-
    solve_equation(A, B, C),
    solve_each_equations(D, E, F).
solve_each_equations([], [], []).

% Asserting clauses for user:solve_each_equations/3


In [68]:
?- solve_each_equations([x + y = 2, x + 3*y = 4], [x, y], Sols).

Sols = [x=2-y,y=(4-x)/3]

他の変数の解を自分の解に代入するため、他の解を抽出するother_eqsを定義します。

In [69]:
other_eqs(X, [Y=Rhs|Eqs], [Y=Rhs|OEqs]) :- X \= Y, other_eqs(X, Eqs, OEqs).
other_eqs(X, [Y=Rhs|Eqs], OEqs) :- X == Y, other_eqs(X, Eqs, OEqs).
other_eqs(X, [], []).

% Asserting clauses for user:other_eqs/3


In [70]:
?- other_eqs(x, [x=2-y,y=(4-x)/3], OEqs).

OEqs = [y=(4-x)/3]

In [71]:
% X以外の解をEqに代入し、Xの解を求める
subs_others([X|Xs], [Eq|Eqs], Sols, [Sol2|SubSols]) :- other_eqs(X, Sols, OSols), substitute(Eq, OSols, SubSol), solve_equation(SubSol, X, Sol2), subs_others(Xs, Eqs, Sols, SubSols).
subs_others(X, [], Sols, []).

% Asserting clauses for user:subs_others/4


In [72]:
?- subs_others([x, y], [x + y = 2, x + 3*y = 4], [x=2-y,y=(4-x)/3], SubSol).

SubSol = [x= - -0.6666666666666667/0.6666666666666667,y= - -2/2]

In [73]:
solve_simultaneous_equations(Eqs, Xs, SubSols) :-
    solve_each_equations(Eqs, Xs, Sols), 
    subs_others(Xs, Eqs, Sols, SubSols).

Previously defined clauses of user:solve_simultaneous_equations/3 were retracted:
:- dynamic solve_simultaneous_equations/3.

solve_simultaneous_equations([A|B], [C|D], [H|J]) :-
    solve_each_equations([A|B],
                         [C|D],
                         [E|F]),
    substitute(E, F, G),
    solve_equation(G, C, H),
    substitue_solution(B, H, I),
    solve_simultaneous_equations(I, D, J).
solve_simultaneous_equations([], [], []).

% Asserting clauses for user:solve_simultaneous_equations/3


In [74]:
?- solve_each_equations([3*x + 4*y = 12, 2*x + 5*y = 7], [x, y], Sols).

Sols = [x=(12-4*y)/3,y=(7-2*x)/5]

In [75]:
?- subs_others([x, y], [3*x + 4*y = 12, 2*x + 5*y = 7], [x=(12-4*y)/3,y=(7-2*x)/5], SubSol).

SubSol = [x= - -6.3999999999999995/1.4,y= - 1.0/2.3333333333333335]

最後に連立方程式の解を出力します。

In [76]:
?- solve_simultaneous_equations([3*x + 4*y = 12, 2*x + 5*y = 7], [x, y], Sols).

Sols = [x= - -6.3999999999999995/1.4,y= - 1.0/2.3333333333333335]

In [77]:
% 右辺の数式を評価し、数値に変換する
numeric_solves([X = Rhs|Eqs], [X|Xs], [X = N|Sols]) :- N is Rhs, numeric_solves(Eqs, Xs, Sols).
numeric_solves([], [], []).

% Asserting clauses for user:numeric_solves/3


In [78]:
?- numeric_solves([x= - -6.3999999999999995/1.4,y= - 1.0/2.3333333333333335], [x, y], Sols).

Sols = [x=4.571428571428571,y= -0.42857142857142855]

In [79]:
?- solve_simultaneous_equations([3*x + 4*y = 12, 2*x + 5*y = 7], [x, y], Sols), numeric_solves(Sols, [x, y], NumSols).

Sols = [x= - -6.3999999999999995/1.4,y= - 1.0/2.3333333333333335],
NumSols = [x=4.571428571428571,y= -0.42857142857142855]

## 式の整形
pressのutilに含まれている本格的な簡素化関数tidyを使うと式がきれいに整形されます。


In [2]:
:- [
    './from_press/util_ops.pl',
    './from_press/long.pl',
    './from_press/tidy.pl'
   ].

ERROR: /workspaces/prolog/from_press/long.pl:402:6: Syntax error: Operator expected
ERROR: /workspaces/prolog/from_press/long.pl:403:6: Syntax error: Operator expected
ERROR: /workspaces/prolog/from_press/tidy.pl:85:7: Syntax error: Operator expected

In [4]:
?- tidy(2/2 - -x, A).

A = 1+x